# Simple notebook with connection example

This notebook should illustrate how to connect to the Neo4j database, query information and transform it into Python Objects for further processing.

In [10]:
# These are jupyter specific settings. Keep these when you import your scripts from a relative folder
%load_ext autoreload
%autoreload 2

%config Application.log_level='INFO'

In [15]:
import pandas as pd
import numpy as np

from neo4j import GraphDatabase

### Connection details

__BE AWARE TO USE THE CORRECT USERNAME AND APPLICATION PORTS!__

In [12]:
uri, user, password = "bolt://localhost:7691", "username", "password"
driver = GraphDatabase.driver(uri, auth=(user, password))

# Queries
### I. Directed and undirected queries for relationships

In [ ]:
with driver.session() as session:
    query = """
        MATCH (n:Node {id:32591, time:300}) RETURN size((n)--())
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)

with driver.session() as session:
    query = """
        MATCH (n:Node {id:32591, time:300}) RETURN size((n)-->())
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)

with driver.session() as session:
    query = """
        MATCH (n:Node {id:32591, time:300}) RETURN size((n)--(:Node))
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)

### II. Query standalone Loops from a specific time as parameter

In [ ]:
with driver.session() as session:
    query = """
        MATCH (n:Loop)
        WHERE n.time = 50
        RETURN n
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)

### III. Query Loops and their direct neighbours

This query is probably not a good idea, as it returns all the Nodes that all the Loops consist of. This results in more Nodes than exist in a single state, because some nodes are assigned to two or more Loops and these duplicates are also returned.

In [ ]:
with driver.session() as session:
    query = """
        MATCH (n:Loop)--(m)
        WHERE n.time = 50
        RETURN distinct n, count(m)
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)

### II. Query Loops and their connected Junctions and neighbour Loops

This following query is better and concentrates only on the "meta nodes" that are connected to a loop. The size of our result set is drastically smaller and the returned neighbours are far more likely to be useful for our collective matching.

In [ ]:
with driver.session() as session:
    query = """
        MATCH (n:Loop)--(m)--(l)
        WHERE n.time = 50 and (l:Loop)
        RETURN n.id, collect(distinct l.id)
        """
    result = session.run(query)
    data = pd.DataFrame(result.data())
print(data)